In [1]:
import cudf as pd
import numpy as np

In [2]:
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    precision_score,
    f1_score,
    accuracy_score,
    recall_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    mean_squared_error,
    mean_absolute_error,
    f1_score,
)
from xgboost import XGBClassifier

train = pd.read_csv(
    "../QS4ML-VU-100/Dataset/Preparation_dir/final_data/feature_engineering_data/version2/train.csv"
)
test = pd.read_csv(
    "../QS4ML-VU-100//Dataset/Preparation_dir/final_data/feature_engineering_data/version2/test.csv"
)

target = "act"

In [3]:

from sklearn.feature_selection import RFE

selected_features_list = [
    "BandAccX",
    "BandAccY",
    "BandAccZ",
    "speed",
    "rate",
    "BandAccX_diff",
    "BandAccY_diff",
    "BandAccZ_diff",
    "speed_diff",
    "rate_diff",
    "rateZone",
    "usage",
    "deviceStatus",
    "Acceleration_X",
    "Acceleration_Y",
    "Acceleration_Z",
    "MagneticField_X",
    "MagneticField_Y",
    "MagneticField_Z",
    "Orientation_X",
    "Orientation_Y",
    "Orientation_Z",
    "AngularVelocity_X",
    "AngularVelocity_Y",
    "AngularVelocity_Z",
    "Acceleration_X_diff",
    "Acceleration_Y_diff",
    "Acceleration_Z_diff",
    "MagneticField_X_diff",
    "MagneticField_Y_diff",
    "MagneticField_Z_diff",
    "Orientation_X_diff",
    "Orientation_Y_diff",
    "Orientation_Z_diff",
    "AngularVelocity_X_diff",
    "AngularVelocity_Y_diff",
    "AngularVelocity_Z_diff",
    "act",
    "Acceleration_X_rolling_min",
    "Acceleration_Y_rolling_min",
    "Acceleration_Z_rolling_min",
    "MagneticField_X_rolling_min",
    "MagneticField_Y_rolling_min",
    "MagneticField_Z_rolling_min",
    "Orientation_X_rolling_min",
    "Orientation_Y_rolling_min",
    "Orientation_Z_rolling_min",
    "AngularVelocity_X_rolling_min",
    "AngularVelocity_Y_rolling_min",
    "AngularVelocity_Z_rolling_min",
    "BandAccX_rolling_min",
    "BandAccY_rolling_min",
    "BandAccZ_rolling_min",
    "speed_rolling_min",
    "rate_rolling_min",
    "Acceleration_X_rolling_max",
    "Acceleration_Y_rolling_max",
    "Acceleration_Z_rolling_max",
    "MagneticField_X_rolling_max",
    "MagneticField_Y_rolling_max",
    "MagneticField_Z_rolling_max",
    "Orientation_X_rolling_max",
    "Orientation_Y_rolling_max",
    "Orientation_Z_rolling_max",
    "AngularVelocity_X_rolling_max",
    "AngularVelocity_Y_rolling_max",
    "AngularVelocity_Z_rolling_max",
    "BandAccX_rolling_max",
    "BandAccY_rolling_max",
    "BandAccZ_rolling_max",
    "speed_rolling_max",
    "rate_rolling_max",
    "Acceleration_X_rolling_std",
    "Acceleration_Y_rolling_std",
    "Acceleration_Z_rolling_std",
    "MagneticField_X_rolling_std",
    "MagneticField_Y_rolling_std",
    "MagneticField_Z_rolling_std",
    "Orientation_X_rolling_std",
    "Orientation_Y_rolling_std",
    "Orientation_Z_rolling_std",
    "AngularVelocity_X_rolling_std",
    "AngularVelocity_Y_rolling_std",
    "AngularVelocity_Z_rolling_std",
    "BandAccX_rolling_std",
    "BandAccY_rolling_std",
    "BandAccZ_rolling_std",
    "speed_rolling_std",
    "rate_rolling_std",
    "Acceleration_X_rolling_mean",
    "Acceleration_Y_rolling_mean",
    "Acceleration_Z_rolling_mean",
    "MagneticField_X_rolling_mean",
    "MagneticField_Y_rolling_mean",
    "MagneticField_Z_rolling_mean",
    "Orientation_X_rolling_mean",
    "Orientation_Y_rolling_mean",
    "Orientation_Z_rolling_mean",
    "AngularVelocity_X_rolling_mean",
    "AngularVelocity_Y_rolling_mean",
    "AngularVelocity_Z_rolling_mean",
    "BandAccX_rolling_mean",
    "BandAccY_rolling_mean",
    "BandAccZ_rolling_mean",
    "speed_rolling_mean",
    "rate_rolling_mean",
    "act"
]

# 只选择selected_features_list中的特征
X_train = train[selected_features_list]
X_test = test[selected_features_list]


In [4]:
import numpy as np
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
from cuml.metrics import confusion_matrix
import itertools


# 获取数据集中所有常数列的列表
constant_columns = [col for col in X_train.columns if X_train[col].nunique() <= 1]

# 删除这些列
X_train = X_train.drop(columns=constant_columns)
X_test = X_test.drop(columns=constant_columns)

# 创建训练集的特征和目标变量
target = "act"  # 需要替换为你的目标列名
y_train = X_train[target]
X_train = X_train.drop(target, axis=1)

# 创建测试集的特征和目标变量
y_test = X_test[target]
X_test = X_test.drop(target, axis=1)

# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# 定义要搜索的超参数网格
param_grid = {
    "n_estimators": [5, 10, 20, 50, 100, 200],
    "max_depth": [5, 10, 15, 20],
    "max_features": ["sqrt", "log2"],
}

# 初始化最佳参数和最高得分
best_params = None
best_score = -np.inf


def f1_score(y_true, y_pred):
    # 计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)

    # 计算精确度和召回率
    precision = cm[1, 1] / (cm[1, 1] + cm[0, 1])
    recall = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # 计算 F1 分数
    f1 = 2 * (precision * recall) / (precision + recall)

    return f1


# 对每一组参数进行网格搜索
for params in itertools.product(
    param_grid["n_estimators"], param_grid["max_depth"], param_grid["max_features"]
):
    # 创建并训练模型
    model = RandomForestClassifier(
        n_estimators=params[0],
        max_depth=params[1],
        max_features=params[2]
    )
    model.fit(X_train, y_train)

    # 在验证集上评估模型
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)

    # 如果这个得分比我们之前找到的得分更好，那么就更新最佳参数和最高得分
    if score > best_score:
        best_score = score
        best_params = {
            "n_estimators": params[0],
            "max_depth": params[1],
            "max_features": params[2],
        }
        best_model = model

# 使用最佳模型计算 F1 分数
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)

print("Best parameters:", best_params)
print("Best accuracy score:", best_score)
print("F1 score of the best model:", f1)

/home/taoyida/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Best parameters: {'n_estimators': 10, 'max_depth': 15, 'max_features': 'sqrt'}
Best accuracy score: 0.42635294795036316
F1 score of the best model: 0.49484997760859833
